In [ ]:
import datetime as dt
import itertools
import inspect
import logging
import matplotlib.pyplot as plt
import operator
import os
import pandas as pd
import pickle
import random
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import json
from random import shuffle

## Data loading and preprocessing

In [ ]:
idc = pd.read_csv('IdcCalcHistory_dist.csv')

In [ ]:
df_sales = pd.read_csv('WarehouseShipmentOperations_dist_21_02to_01_03.csv')

## Create warehouse topology


In [ ]:
class Product:
    def __init__(self, id, volume_liters, idc, coi, coi_with_ads, exemplar_id):
        self.id = id
        self.volume_liters = volume_liters
        self.idc = idc
        self.coi = coi
        self.coi_with_ads = coi_with_ads
        self.exemplar_id = exemplar_id

In [ ]:
class Shelf:
    def __init__(self, max_volume_liters):
        self.max_volume_liters = max_volume_liters
        self.products = []

    def add_product(self, product):
        if self.check_fit(product):
            self.products.append(product)

    def remove_product(self, product_to_remove):
        to_rem = [product for product in self.products if (product_to_remove.id == product.id and product_to_remove.exemplar_id == product.exemplar_id)]
        self.products.remove(to_rem[0])

    def check_fit(self, product):
        total_volume = sum([p.volume_liters for p in self.products]) + product.volume_liters
        return total_volume <= self.max_volume_liters

    def occupancy_ratio(self):
        total_volume = sum([p.volume_liters for p in self.products])
        return total_volume / self.max_volume_liters

In [ ]:
class Warehouse:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        self.warehouse_map = [[None] * columns for _ in range(rows)]

    def add_shelf(self, row, column, max_volume_liters):
        if row < self.rows and column < self.columns:
            self.warehouse_map[row][column] = Shelf(max_volume_liters)

    def remove_shelf(self, row, column):
        if row < self.rows and column < self.columns:
            self.warehouse_map[row][column] = None

    def visualize_warehouse(self, visualization_type):
        if visualization_type not in ['occupancy', 'idc', 'coi']:
            raise ValueError("Invalid visualization type. Must be one of: 'occupancy', 'idc', 'coi'")

        warehouse_matrix = np.zeros((self.rows, self.columns))
        for i in range(self.rows):
            for j in range(self.columns):
                if self.warehouse_map[i][j]:
                    if visualization_type == 'occupancy':
                        warehouse_matrix[i][j] = 1 - self.warehouse_map[i][j].occupancy_ratio()
                    elif visualization_type == 'idc':
                        if self.warehouse_map[i][j].products:
                            avg_idc = sum([product.idc for product in self.warehouse_map[i][j].products]) / len(self.warehouse_map[i][j].products)
                            warehouse_matrix[i][j] = avg_idc
                        else:
                            warehouse_matrix[i][j] = -1
                    elif visualization_type == 'coi':
                        if self.warehouse_map[i][j].products:
                            avg_coi = sum([product.coi for product in self.warehouse_map[i][j].products]) / len(self.warehouse_map[i][j].products)
                            warehouse_matrix[i][j] = avg_coi
                        else:
                            warehouse_matrix[i][j] = -1
                else:
                    warehouse_matrix[i][j] = np.nan

        plt.figure(figsize=(20,20))
        
        if visualization_type == 'occupancy':
            plt.imshow(warehouse_matrix, cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=1)
            plt.colorbar(label='Occupancy Ratio')
        elif visualization_type in ['idc', 'coi']:
            plt.imshow(warehouse_matrix, cmap='RdYlGn', interpolation='nearest')
            plt.colorbar(label='Average Value')
        plt.title('Warehouse Contents')
        plt.xlabel('Columns')
        plt.ylabel('Rows')
        plt.xticks(range(self.columns))
        plt.yticks(range(self.rows))
        plt.grid(True)
        plt.show()
        
    def find_closest_item_to_point(self, itemId, row, column, routing_policy):
        min_distance = float('inf')
        nearest_product_coordinates = None
        
        for i in range(self.rows):
            for j in range(self.columns):
                if self.warehouse_map[i][j] and self.warehouse_map[i][j].products:
                    for product in self.warehouse_map[i][j].products:
                        if product.id == itemId:
                            distance = abs(row - i) + abs(column - j)
                            if distance < min_distance:
                                min_distance = distance
                                nearest_product_coordinates = (i, j)
        
        return nearest_product_coordinates

    
    def visualize_product_location(self, products):
            locations = list()
            for item in products:
                location = self.find_product_location(item.id, item.exemplar_id)
                locations.append(location)

            print(locations)
            warehouse_matrix = np.zeros((self.rows, self.columns))

            for i in range(self.rows):
                for j in range(self.columns):
                    if self.warehouse_map[i][j]:
                        if (i, j) in locations:
                            warehouse_matrix[i][j] = 100
                        else:
                            warehouse_matrix[i][j] = 0
                    else:
                        warehouse_matrix[i][j] = np.nan

            plt.figure(figsize=(20,20))
        
            plt.imshow(warehouse_matrix, cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=100)
            plt.colorbar(label='Occupancy Ratio')
            plt.title('Warehouse Contents')
            plt.xlabel('Columns')
            plt.ylabel('Rows')
            plt.xticks(range(self.columns))
            plt.yticks(range(self.rows))
            plt.grid(True)
            plt.show()
            
    def visualize_specific_locations(self, locations):
            warehouse_matrix = np.zeros((self.rows, self.columns))

            for i in range(self.rows):
                for j in range(self.columns):
                    if self.warehouse_map[i][j]:
                        if (i, j) in locations:
                            warehouse_matrix[i][j] = 100
                        else:
                            warehouse_matrix[i][j] = 0
                    else:
                        warehouse_matrix[i][j] = np.nan

            plt.figure(figsize=(20,20))
            plt.imshow(warehouse_matrix, cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=100)
            plt.colorbar(label='Occupancy Ratio')
            plt.title('Warehouse Contents')
            plt.xlabel('Columns')
            plt.ylabel('Rows')
            plt.xticks(range(self.columns))
            plt.yticks(range(self.rows))
            plt.grid(True)
            plt.show()

    def find_product_location(self, item_id, exemplar_id):
        for i in range(self.rows):
            for j in range(self.columns):
                if self.warehouse_map[i][j] and self.warehouse_map[i][j].products:
                    for product in self.warehouse_map[i][j].products:
                        if product.id == item_id and product.exemplar_id == exemplar_id:
                            return i, j 
        return None 
    
    def visualize_picking_route(self, route, item_locations):
        warehouse_matrix = np.zeros((self.rows, self.columns))
        plt.figure(figsize=(20,20))
        for i in range(len(route) - 1):
            current_location = route[i]
            next_location = route[i + 1]
            x1, y1 = current_location
            x2, y2 = next_location
            plt.plot([y1, y2], [x1, x2], color='blue', linewidth=2)

        for i in range(self.rows):
            for j in range(self.columns):
                if self.warehouse_map[i][j]:                    
                    warehouse_matrix[i][j] = 0
                    if (i, j) in item_locations:
                        warehouse_matrix[i][j] = 100
                else:
                    warehouse_matrix[i][j] = np.nan
                    
        plt.imshow(warehouse_matrix, cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=100)
        plt.colorbar(label='Occupancy Ratio')
        plt.title('Picking Route')
        plt.xlabel('Columns')
        plt.ylabel('Rows')
        plt.xticks(range(self.columns))
        plt.yticks(range(self.rows))
        plt.grid(True)
        plt.show()

    def L1_distance(self, point1, point2): ## L1 dist
        x1, y1 = point1
        x2, y2 = point2
        return abs(x2 - x1) + abs(y2 - y1)
    
    def perform_order_pick(self, routing_policy, locations_with_items, visualize):
        counter = 0
        route = []  
        locations = [o[0] for o in locations_with_items]
        for product_with_locations in locations_with_items:
            product = product_with_locations[1]
            location = product_with_locations[0]
            self.warehouse_map[location[0]][location[1]].remove_product(product)

        if(routing_policy == 's_shape'):
            locations = sorted(locations, key = lambda x: x[1])
            route.append((0,0))
            visited_columns = set()
            on_other_side = False
            for location in locations:
                target_column = location[1]
                if target_column in visited_columns:
                    continue
                visited_columns.add(target_column)
                if(self.warehouse_map[2][target_column+1] is not None):
                    visited_columns.add(target_column - 2)
                    target_column = target_column - 1
                else:
                    target_column = target_column + 1
                    visited_columns.add(target_column + 2)

                if(on_other_side):
                    route.append((18, target_column))
                    route.append((0, target_column))
                    on_other_side = False
                else:
                    route.append((0, target_column))
                    route.append((18, target_column))
                    on_other_side = True
            
            if(on_other_side):
                col = locations[len(locations)-1][1]
                if(self.warehouse_map[2][col+1] is not None):
                    col = col - 1
                else:
                    col = col + 1
                route.append((18, col))
                route.append((18, 0))
                
            route.append((0, 0))
            if(visualize == True):
                print(route)  
                self.visualize_picking_route(route, locations)
            for i in range(len(route) - 1):
                counter += self.L1_distance(route[i], route[i+1]) 
            return counter

        if(routing_policy == 'return'): 
            locations = sorted(locations, key = lambda x: x[1])
            route.append((0,0))
            visited_columns = set()
            for location in locations:
                target_column = location[1]
                if target_column in visited_columns:
                    continue
                visited_columns.add(target_column)
                max_x = float('-inf') 
                if(self.warehouse_map[2][target_column+1] is not None):
                    visited_columns.add(target_column - 2)
                    target_column = target_column - 1
                    for x, y in locations:
                        if y == target_column + 1 and x > max_x:
                            max_x = x
                else:
                    target_column = target_column + 1
                    visited_columns.add(target_column + 2)
                    for x, y in locations:
                        if y == target_column - 1 and x > max_x:
                            max_x = x

                for x, y in locations:
                    if y == target_column and x > max_x:
                        max_x = x

                route.append((0, target_column))
                route.append((max_x, target_column))
                route.append((0, target_column))
            
            col = locations[len(locations)-1][1]
            if(self.warehouse_map[2][col+1] is not None):
                col = col - 1
            else:
                col = col + 1
            route.append((0, col))
            route.append((0,0))
            if(visualize == True):
                print(route)  
                self.visualize_picking_route(route, locations)
            for i in range(len(route) - 1):
                counter += self.L1_distance(route[i], route[i+1]) 

        return counter

    def place_product(self, product, routing_policy):
        if(routing_policy == 's_shape'):
            for col in range(1,41):
                for row in range(2, 18):
                    if(col % 3 != 0):
                        if self.warehouse_map[row][col] is not None and self.warehouse_map[row][col].check_fit(product):
                            shelf = self.warehouse_map[row][col]
                            shelf.add_product(product)
                            return
        elif(routing_policy == 'return'):
            for row in range(2,18):
                for col in range(1, 41):
                    if(col % 3 != 0):
                        if self.warehouse_map[row][col] is not None and self.warehouse_map[row][col].check_fit(product):
                            shelf = self.warehouse_map[row][col]
                            shelf.add_product(product)
                            return
        elif(routing_policy == 'random'):
            while(True):
                row = random.randint(2, 18)
                col = random.randint(1, 41)
                if(col % 3 != 0):
                    if self.warehouse_map[row][col] is not None and self.warehouse_map[row][col].check_fit(product):
                        shelf = self.warehouse_map[row][col]
                        shelf.add_product(product)
                        return           

In [ ]:
def create_warehouse(asile_lenght = 1, asile_size = 1, asile = 1):
    warehouse = Warehouse(20, 42)

    for row in range(2,18):
        for col in range(1, 41):
            if col % 3 == 1 or col % 3 == 2:
                warehouse.add_shelf(row, col, 5)
    return warehouse

In [ ]:
total_number_of_sales = 0
for index, row in idc_1.iterrows():
        total_number_of_sales += row['AdsNum'] * row['StockAvailDays']
        

In [ ]:
total_number_of_sales

In [ ]:
def get_product_list():
    product_list = list() 
    for index, row in idc_1.iterrows():
            curr_stock = 1
            curr_volume = row['Volume']
            for i in range(row['CurrentStock']):
                product = Product(id=row['ItemId'], volume_liters=row['Volume'], idc=curr_stock/row['AdsNum'], coi = (row['Volume']*row['CurrentStock'])/((row['AdsNum'] * row['StockAvailDays'])/28), coi_with_ads = (curr_volume)/(row['AdsNum']), exemplar_id = i)
                product_list.append(product)
                curr_stock += 1
                curr_volume += curr_volume
    return product_list

In [ ]:
def get_exemplars_order(products, item_id, already_taken_product, metric):
    filtered_products = [product for product in products if product.id == item_id and (product.id, product.exemplar_id) not in already_taken_product]
    
    if not filtered_products:
        return None
    if(metric == 'idc'):
        lowest_idc_product = filtered_products[0]
        for product in filtered_products:
            if product.idc < lowest_idc_product.idc:
                lowest_idc_product = product
        return lowest_idc_product    
    if(metric == 'coi'):
        lowest_coi_product = filtered_products[0]
        for product in filtered_products:
            if product.coi < lowest_coi_product.coi:
                lowest_coi_product = product
        return lowest_coi_product
    if(metric == 'random'):
        return random.choice(filtered_products)    

In [ ]:
product_list = get_product_list()

## Shared

In [ ]:
statistics = {}

In [ ]:
period_statistics = {}

## IDC s_shape

In [ ]:
warehouse = create_warehouse()

In [ ]:
product_list = sorted(product_list, key = lambda x: x.idc)


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
product_ids = [p.id for p in product_list]

unique_product_ids = list(set(product_ids))
df_sales_1 = orders_to_take[orders_to_take['RezonItemId'].isin(unique_product_ids)]


In [ ]:
df_sales_1 = df_sales_1[df_sales_1['Quantity'] == 1]

In [ ]:
df_sales_1 = df_sales_1[df_sales_1['Quantity'] == 1]

In [ ]:
df_sales_period['Quantity'] = 1

In [ ]:
df_sales_period

In [ ]:
unique_orders = df_sales_1['OmsPostingId'].unique()

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для IDC")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега s-shape пробегов за период 30 дней размещения модели для IDC")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'IDC s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
order_items

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
warehouse.visualize_warehouse('occupancy')

## IDC return

In [ ]:
warehouse = create_warehouse()

In [ ]:
product_list = sorted(product_list, key = lambda x: x.idc)


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для IDC")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для IDC")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'IDC return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat) , 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
warehouse.visualize_warehouse('occupancy')

## COI s_shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
df_sales_1

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для COI")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'COI s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

## COI return shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для COI")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            if(exmp is None):
                continue
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## COI with ads s-shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi_with_ads)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для COI with ads")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI with ads s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI with ads s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat) , 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## COI with ads return shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi_with_ads)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для COI")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI with ads return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI with ads return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Random return

In [ ]:
warehouse = create_warehouse()

shuffle(product_list)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для randoim")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'random')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            exemplars_in_order.append(exmp)
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для Random")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'random return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['Random return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Random s-shape

In [ ]:
warehouse = create_warehouse()


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'random')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для random")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'random')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега s_shape пробегов за период 30 дней размещения модели для Random")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue        
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'random s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
unique_dates

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['random s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Statistics

In [ ]:
for metric, stats in statistics.items():

    print(f"Метрика: {metric}")
    print("Статистики:")
    for stat_name, value in stats.items():
        if(metric == 'order_dist'):
            continue
        print(f"{stat_name}: {value}")
    print()

In [ ]:
s_shape_metrics = {key: value for key, value in statistics.items() if 's-shape' in key}

# Построить графики для метрик s-shape
plt.figure(figsize=(50, 9))
for key, value in s_shape_metrics.items():
    order_dist = value['order_dist']
    num_items = len(order_dist)
    indices = np.arange(1, num_items + 1)
    plt.plot(indices, order_dist,  linestyle='-', linewidth = 4, label=key)

plt.xlabel('Index')
plt.ylabel('Order Distance')
plt.title(' ')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
s_shape_metrics = {key: value for key, value in statistics.items() if 'return' in key}

# Построить графики для метрик s-shape
plt.figure(figsize=(50, 9))
for key, value in s_shape_metrics.items():
    order_dist = value['order_dist']
    num_items = len(order_dist)
    indices = np.arange(1, num_items + 1)
    plt.plot(indices, order_dist,  linestyle='-', linewidth = 4, label=key)

plt.xlabel('Index')
plt.ylabel('Order Distance')
plt.title('Количество пробегов для одного и того же заказа с эвристикой "return" в разрезе статистик')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
len(period_statistics)

In [ ]:
# Перебор данных в period_statistics для каждой даты
for (date, metric), stats_df in period_statistics.items():
    print(f"Дата: {date}")
    print(f"Метрика: {metric}")
    print("Статистики:")
    # Перебор метрик для данной даты
    for stat_name, value in stats_df.items():
        if metric == 'order_dist':
            continue
        print(f"{stat_name}: {value}")



In [ ]:
len(period_statistics)

In [ ]:
# Создаем словарь для хранения средних значений для метрик, содержащих 's_shape' в названии
mean_s_shape_metrics = {metric: [] for (date, metric) in period_statistics.keys() if 's_shape' in metric}
unique_dates = set()

# Перебираем данные в period_statistics
for (date, metric), stats_df in period_statistics.items():
    if 's_shape' in metric:
        unique_dates.add(date)
        mean_value = stats_df['mean']  # Предполагается, что 'mean' - это название столбца со средними значениями
        mean_s_shape_metrics[metric].append(mean_value)

dates = sorted(list(unique_dates))
# Строим график для каждой метрики, содержащей 's_shape' в названии
plt.figure(figsize=(10, 6))
for metric, values in mean_s_shape_metrics.items():
    if values:  # Проверяем, есть ли значения для данной метрики
        plt.plot(dates, values, label=metric)

plt.xlabel('Дата')
plt.ylabel('Среднее значение')
plt.title('Средние значения метрик с s_shape по датам')
plt.xticks(rotation=45)  # Поворачиваем подписи по оси X для лучшей читаемости
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

